# Stock News Analysis with Amazon Bedrock

This notebook gathers news articles for a given stock over the last week, selects the 10 most interesting ones for predicting stock price evolution, and assigns scores for positivity, impact, and time scale using Amazon Bedrock.

In [4]:
!pip install newsapi-python

In [14]:
import boto3
import json
from datetime import datetime, timedelta
from newsapi import NewsApiClient
import pandas as pd
import os

In [6]:
# Configuration
NEWS_API_KEY = 'c36b6af513a54b1486d9ee565e36b2a3'  # Replace with your NewsAPI key
AWS_REGION = 'us-east-1'  # Replace with your AWS region
BEDROCK_MODEL_ID = 'anthropic.claude-3-sonnet-20240229-v1:0'  # Or another model like 'anthropic.claude-3-sonnet-20240229-v1:0'
STOCK_TICKER = 'Tesla'  # Replace with the stock ticker you want to analyze
TOP_N_NEWS = 10

# Initialize clients
newsapi = NewsApiClient(api_key=NEWS_API_KEY)
bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)

In [ ]:
# def fetch_news(stock_ticker, days_back=7):
#     """
#     Fetch news articles for a given stock over the last N days.
#     """
#     end_date = datetime.now()
#     start_date = end_date - timedelta(days=days_back)
    
#     query = f'"{stock_ticker}" stock OR "{stock_ticker}" shares OR "{stock_ticker}" price'
    
#     all_articles = newsapi.get_everything(
#         q=query,
#         from_param=start_date.strftime('%Y-%m-%d'),
#         to=end_date.strftime('%Y-%m-%d'),
#         language='en',
#         sort_by='relevancy',
#         page_size=100  # Max per page
#     )
    
#     articles = all_articles['articles']
#     news_items = []
#     for article in articles:
#         news_items.append({
#             'title': article['title'],
#             'description': article['description'],
#             'content': article.get('content', ''),
#             'url': article['url'],
#             'publishedAt': article['publishedAt'],
#             'source': article['source']['name']
#         })
    
#     return news_items

In [ ]:
# def call_bedrock(prompt, model_id=BEDROCK_MODEL_ID):
#     """
#     Call Amazon Bedrock with a prompt and return the response.
#     """
#     if 'anthropic' in model_id.lower():
#         # Claude format
#         body = json.dumps({
#             "messages": [
#                 {
#                     "role": "user",
#                     "content": prompt
#                 }
#             ],
#             "max_tokens": 1000,
#             "temperature": 0.7,
#             "top_p": 0.9,
#             "anthropic_version": "bedrock-2023-05-31"
#         })
#     else:
#         # Titan format
#         body = json.dumps({
#             "inputText": prompt,
#             "textGenerationConfig": {
#                 "maxTokenCount": 1000,
#                 "temperature": 0.7,
#                 "topP": 0.9
#             }
#         })
    
#     response = bedrock_runtime.invoke_model(
#         modelId=model_id,
#         body=body
#     )
    
#     response_body = json.loads(response['body'].read())
    
#     if 'anthropic' in model_id.lower():
#         return response_body['content'][0]['text']
#     else:
#         return response_body['results'][0]['outputText']

In [ ]:
# def select_interesting_news(news_items, stock_ticker, top_n=10):
#     """
#     Use Bedrock to select the top N most interesting news articles for stock price prediction.
#     """
#     news_list = "\n".join([f"{i+1}. {item['title']}: {item['description']}" for i, item in enumerate(news_items)])
    
#     prompt = f"""
#     Given the following news articles about {stock_ticker} stock, select the {top_n} most interesting ones that could directly impact the stock price evolution.
    
#     Selection criteria:
#     - Only select articles that are specifically about {stock_ticker} company or its direct operations.
#     - Focus on company-specific news like earnings reports, financial results, mergers & acquisitions, product launches, executive changes, legal issues, or major business developments that affect {stock_ticker}.
#     - Do NOT select articles that merely mention {stock_ticker} in passing, in lists of stocks, or as part of broader market discussions without specific relevance to {stock_ticker}.
#     - Articles about general market trends, sector news, or other companies should be ignored unless they directly involve {stock_ticker}.
    
#     News articles:
#     {news_list}
    
#     Return only the numbers of the selected articles (e.g., 1,3,5) in order of interest, separated by commas. If fewer than {top_n} qualify, return only those that meet the criteria.
#     """
    
#     response = call_bedrock(prompt)
#     selected_indices = [int(x.strip()) - 1 for x in response.split(',') if x.strip().isdigit()]
#     selected_news = [news_items[i] for i in selected_indices if i < len(news_items)]
#     return selected_news[:top_n]

In [ ]:
# def score_article(article, stock_ticker):
#     """
#     Use Bedrock to assign scores for positivity, impact, and time scale.
#     """
#     prompt = f"""
#     Analyze the following news article. You will need to undestand what it says about {stock_ticker} stock and assign scores out of 10 for:
#     1. Positivity: How positive is the news for the stock price? (0 = stock price will decrease, 10 = stock price will increase)
#     2. Impact: How big is the potential impact on the stock price? (0 = little impact, 10 = very big impact)
#     3. Time scale: How long-term is the effect? (0 = very long term effect, 10 = very short term effect/already happened)
    
#     Article title: {article['title']}
#     Article description: {article['description']}
#     Article content: {article['content']}
    
#     Return only a JSON object with keys 'positivity', 'impact', 'time_scale' and their scores as numbers.
#     """
    
#     response = call_bedrock(prompt)
#     try:
#         scores = json.loads(response)
#         return scores
#     except:
#         # Fallback if JSON parsing fails
#         return {'positivity': 5, 'impact': 5, 'time_scale': 5}

In [ ]:
# def explain_score(article, scores, stock_ticker):
#     """
#     Use Bedrock to generate a short explanation of the scores assigned to the article.
#     """
#     prompt = f"""
#     Based on the following news article about {stock_ticker} stock, you previously assigned the following scores:
#     - Positivity: {scores['positivity']}/10
#     - Impact: {scores['impact']}/10
#     - Time Scale: {scores['time_scale']}/10
    
#     Provide a short explanation (no more than 2 sentences) about these scores. It must convey what drove the scoring decision but avoid repeating the scores themselves. It be very concise.
    
#     Article title: {article['title']}
#     Article description: {article['description']}
#     Article content: {article['content']}
    
#     Explanation:
#     """
    
#     response = call_bedrock(prompt)
#     return response.strip()

In [ ]:
# # Fetch news
# news_items = fetch_news(STOCK_TICKER)
# print(f"Fetched {len(news_items)} news articles for {STOCK_TICKER}")

# # Select interesting news
# interesting_news = select_interesting_news(news_items, STOCK_TICKER, TOP_N_NEWS)
# print(f"Selected {len(interesting_news)} most interesting articles")

# # Score each article
# scored_news = []
# for article in interesting_news:
#     scores = score_article(article, STOCK_TICKER)
#     explanation = explain_score(article, scores, STOCK_TICKER)
#     scored_article = {**article, **scores, 'explanation': explanation}
#     scored_news.append(scored_article)

# # Create DataFrame for display
# df = pd.DataFrame(scored_news, columns=['title', 'description', 'positivity', 'impact', 'time_scale', 'explanation', 'url', 'publishedAt', 'source'])
# df

Fetched 99 news articles for Tesla
Selected 9 most interesting articles


,title,description,positivity,impact,time_scale,explanation,url,publishedAt,source
0,It's 'Going to Be Like a Shockwave' When Tesla...,Tesla stock has surged to a fresh all-time hig...,9,8,4,The article conveys a highly positive outlook ...,https://www.barchart.com/story/news/35789416/i...,2025-10-30T11:30:02Z,Barchart.com
1,Optimus Could Be ‘the Biggest Product of All T...,Elon Musk’s bold claim that Tesla’s Optimus ro...,7,8,4,The article discusses Tesla's ambitious plans ...,https://www.barchart.com/story/news/35700692/o...,2025-10-27T14:28:36Z,Barchart.com
2,‘We Are at a Critical Inflection Point for Tes...,Tesla’s Q3 results mark a pivotal shift as Mus...,5,8,4,The article highlights Tesla's pivotal strateg...,https://www.barchart.com/story/news/35707602/w...,2025-10-27T19:04:45Z,Barchart.com
3,"Tesla’s New Focus Isn’t on Cars, But on ‘Susta...",Tesla’s new “sustainable abundance” push pivot...,8,7,4,The article's optimistic tone about Tesla's am...,https://www.barchart.com/story/news/35831261/t...,2025-10-31T16:39:56Z,Barchart.com
4,"Freedom Capital Upgrades Tesla (TSLA) to Hold,...",Tesla Inc. (NASDAQ:TSLA) is one of the stocks ...,7,6,5,The positive outlook on Tesla's potential stoc...,https://finance.yahoo.com/news/freedom-capital...,2025-10-29T15:25:04Z,Yahoo Entertainment
5,Bank of America Raises Tesla (TSLA) PT to $471...,"Tesla, Inc. (NASDAQ:TSLA) is one of the AI Sto...",7,6,7,The positivity score reflects the optimistic o...,https://finance.yahoo.com/news/bank-america-ra...,2025-10-31T23:38:35Z,Yahoo Entertainment
6,"Freedom Capital Upgrades Tesla (TSLA) to Hold,...",Tesla Inc. (NASDAQ:TSLA) is one of the stocks ...,7,6,4,"The upgrade to a ""Hold"" rating and increased p...",https://biztoc.com/x/fda6960d84123d13,2025-10-29T15:42:53Z,Biztoc.com
7,Jim Cramer Backs Elon Musk's $1 Trillion Packa...,Renowned TV host Jim Cramer weighed in on Tesl...,8,7,5,The high positivity score reflects Cramer's en...,https://biztoc.com/x/7dc36a43628931d4,2025-10-27T04:57:30Z,Biztoc.com
8,Tesla European Sales Slump Again. Why the Stoc...,,8,6,5,The article's positive tone about Tesla's stoc...,https://biztoc.com/x/38906754ea72f089,2025-10-28T14:19:05Z,Biztoc.com


In [ ]:
# # Display results
# for i, row in df.iterrows():
#     print(f"\n{i+1}. {row['title']}")
#     print(f"   Positivity: {row['positivity']}/10")
#     print(f"   Impact: {row['impact']}/10")
#     print(f"   Time Scale: {row['time_scale']}/10")
#     print(f"   Explanation: {row['explanation']}")
#     print(f"   Source: {row['source']} | Published: {row['publishedAt']}")
#     print(f"   URL: {row['url']}")


1. It's 'Going to Be Like a Shockwave' When Tesla's AI Innovations Hit. Should You Buy TSLA Stock First?
   Positivity: 9/10
   Impact: 8/10
   Time Scale: 4/10
   Explanation: The article conveys a highly positive outlook for Tesla's future growth driven by its AI innovations, justifying the high positivity score. However, the time scale score is lower as the transformative impact of these innovations is expected to unfold gradually over time. The substantial content detailing Tesla's current performance and future potential warrants a high impact score.
   Source: Barchart.com | Published: 2025-10-30T11:30:02Z
   URL: https://www.barchart.com/story/news/35789416/it-s-going-to-be-like-a-shockwave-when-tesla-s-ai-innovations-hit-should-you-buy-tsla-stock-first

2. Optimus Could Be ‘the Biggest Product of All Time.’ Does That Make TSLA Stock a Buy Despite Musk’s Distractions, Tesla’s Earnings Miss?
   Positivity: 7/10
   Impact: 8/10
   Time Scale: 4/10
   Explanation: The article disc

## Explanation of Scores

- **Positivity**: Indicates the sentiment of the news towards the stock price.
  - 0: Strongly negative, likely to decrease stock price
  - 5: Neutral
  - 10: Strongly positive, likely to increase stock price

- **Impact**: Measures the potential magnitude of the effect on the stock price.
  - 0: Minimal impact
  - 5: Moderate impact
  - 10: Major impact

- **Time Scale**: Indicates how quickly the effect might be realized.
  - 0: Long-term effect (months/years)
  - 5: Medium-term (weeks/months)
  - 10: Short-term or immediate effect (days or already occurred)

In [ ]:
def get_news_for_company(company_name):
    # Configuration
    NEWS_API_KEY = 'c36b6af513a54b1486d9ee565e36b2a3'  # Replace with your NewsAPI key
    AWS_REGION = 'us-east-1'  # Replace with your AWS region
    BEDROCK_MODEL_ID = 'us.anthropic.claude-sonnet-4-5-20250929-v1:0'  # Or another model like 'anthropic.claude-3-sonnet-20240229-v1:0'
    TOP_N_NEWS = 10

    # Initialize clients
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)
    bedrock_runtime = boto3.client('bedrock-runtime', region_name=AWS_REGION)

    def fetch_news(stock_ticker, days_back=7):
        """
        Fetch news articles for a given stock over the last N days.
        """
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)
        
        query = f'"{stock_ticker}" stock OR "{stock_ticker}" shares OR "{stock_ticker}" price'
        
        all_articles = newsapi.get_everything(
            q=query,
            from_param=start_date.strftime('%Y-%m-%d'),
            to=end_date.strftime('%Y-%m-%d'),
            language='en',
            sort_by='relevancy',
            page_size=100  # Max per page
        )
        
        articles = all_articles['articles']
        news_items = []
        for article in articles:
            news_items.append({
                'title': article['title'],
                'description': article['description'],
                'content': article.get('content', ''),
                'url': article['url'],
                'publishedAt': article['publishedAt'],
                'source': article['source']['name']
            })
        
        return news_items

    def call_bedrock(prompt, model_id=BEDROCK_MODEL_ID):
        """
        Call Amazon Bedrock with a prompt and return the response.
        """
        if 'anthropic' in model_id.lower():
            # Claude format
            body = json.dumps({
                "messages": [
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                "max_tokens": 1000,
                # "temperature": 0.7,
                # "top_p": 0.9,
                "anthropic_version": "bedrock-2023-05-31"
            })
        else:
            # Titan format
            body = json.dumps({
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 1000,
                    "temperature": 0.7,
                    "topP": 0.9
            }
        })
        
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=body
        )
        
        response_body = json.loads(response['body'].read())
        
        if 'anthropic' in model_id.lower():
            return response_body['content'][0]['text']
        else:
            return response_body['results'][0]['outputText']

    def select_interesting_news(news_items, stock_ticker, top_n=10):
        """
        Use Bedrock to select the top N most interesting news articles for stock price prediction.
        """
        news_list = "\n".join([f"{i+1}. {item['title']}: {item['description']}" for i, item in enumerate(news_items)])
        
        prompt = f"""
        Given the following news articles about {stock_ticker} stock, select the {top_n} most interesting ones that could directly impact the stock price evolution.
        
        Selection criteria:
        - Only select articles that are specifically about {stock_ticker} company or its direct operations.
        - Focus on company-specific news like earnings reports, financial results, mergers & acquisitions, product launches, executive changes, legal issues, or major business developments that affect {stock_ticker}.
        - Do NOT select articles that merely mention {stock_ticker} in passing, in lists of stocks, or as part of broader market discussions without specific relevance to {stock_ticker}.
        - Articles about general market trends, sector news, or other companies should be ignored unless they directly involve {stock_ticker}.
        
        News articles:
        {news_list}
        
        Return only the numbers of the selected articles (e.g., 1,3,5) in order of interest, separated by commas. If fewer than {top_n} qualify, return only those that meet the criteria.
        """
        
        response = call_bedrock(prompt)
        selected_indices = [int(x.strip()) - 1 for x in response.split(',') if x.strip().isdigit()]
        selected_news = [news_items[i] for i in selected_indices if i < len(news_items)]
        return selected_news[:top_n]

    def score_article(article, stock_ticker):
        """
        Use Bedrock to assign scores for positivity, impact, and time scale.
        """
        prompt = f"""
        Analyze the following news article. You will need to undestand what it says about {stock_ticker} stock and assign scores out of 10 for:
        1. Positivity: How positive is the news for the stock price? (0 = stock price will decrease, 10 = stock price will increase)
        2. Impact: How big is the potential impact on the stock price? (0 = little impact, 10 = very big impact)
        3. Time scale: How long-term is the effect? (0 = very long term effect, 10 = very short term effect/already happened)
        
        Article title: {article['title']}
        Article description: {article['description']}
        Article content: {article['content']}
        
        Return only a JSON object with keys 'positivity', 'impact', 'time_scale' and their scores as numbers.
        """
        
        response = call_bedrock(prompt)
        try:
            # remove first and last lines if it starts with ```json
            if response.startswith('```json'):
                response = '\n'.join(response.split('\n')[1:-1])
            scores = json.loads(response)
            return scores
        except:
            # Fallback if JSON parsing fails
            return {'positivity': 5, 'impact': 5, 'time_scale': 5}

    def explain_score(article, scores, stock_ticker):
        """
        Use Bedrock to generate a short explanation of the scores assigned to the article.
        """
        prompt = f"""
        Based on the following news article about {stock_ticker} stock, you previously assigned the following scores:
        - Positivity: {scores['positivity']}/10
        - Impact: {scores['impact']}/10
        - Time Scale: {scores['time_scale']}/10
        
        Provide a short explanation (no more than 2 sentences) about these scores. It must convey what drove the scoring decision but avoid repeating the scores themselves. It be very concise.
        
        Article title: {article['title']}
        Article description: {article['description']}
        Article content: {article['content']}
        
        Explanation:
        """
        
        response = call_bedrock(prompt)
        return response.strip()

    # Fetch news
    news_items = fetch_news(company_name)

    # Select interesting news
    interesting_news = select_interesting_news(news_items, company_name, TOP_N_NEWS)

    # Score each article
    result = []
    for article in interesting_news:
        scores = score_article(article, company_name)
        explanation = explain_score(article, scores, company_name)
        score_value = scores['positivity'] * scores['impact'] / 20
        confiance = scores['impact'] * scores['time_scale'] / 20
        result.append({
            "nom": article['title'],
            "score": score_value,
            "résumé": explanation,
            "lien": article['url'],
            "confiance": confiance
        })
    return result

In [ ]:
# get_news_for_company("Apple")

```json
{
  "positivity": 7,
  "impact": 7,
  "time_scale": 9
}
```
```json
{
  "positivity": 7,
  "impact": 6,
  "time_scale": 9
}
```
```json
{
  "positivity": 8,
  "impact": 6,
  "time_scale": 7
}
```
```json
{
  "positivity": 8,
  "impact": 7,
  "time_scale": 9
}
```
```json
{
  "positivity": 8,
  "impact": 7,
  "time_scale": 9
}
```
```json
{
  "positivity": 4,
  "impact": 3,
  "time_scale": 9
}
```
```json
{
  "positivity": 8,
  "impact": 6,
  "time_scale": 8
}
```
```json
{
  "positivity": 8,
  "impact": 6,
  "time_scale": 8
}
```
```json
{
  "positivity": 7,
  "impact": 5,
  "time_scale": 8
}
```
```json
{
  "positivity": 7,
  "impact": 6,
  "time_scale": 8
}
```


[{'nom': "'Getting its groove back': Wall Street expects strong Q4 earnings for Apple as iPhone demand overrides China worries",
  'score': 2.45,
  'résumé': "The article highlights Wall Street's optimistic outlook on Apple's Q4 earnings driven by strong iPhone demand despite China concerns, and notes the company's recent milestone of surpassing $4 trillion valuation, which explains the moderately positive sentiment and significant market impact. The imminent Thursday earnings report creates clear short-term relevance and urgency for investors.",
  'lien': 'https://www.businessinsider.com/apple-q4-earnings-report-preview-beat-iphone-demand-2025-10',
  'confiance': 3.15},
 {'nom': 'Apple Reports Strong Q4 2025 Results',
  'score': 2.1,
  'résumé': 'The strong quarterly results indicate solid company performance, driving high positivity, while the moderate impact reflects that quarterly earnings are largely expected events. The short time scale stems from quarterly results having immedia